In [1]:
import moxing as mox
mox.file.copy_parallel('obs://deep-learning-hw2-zzzzzzjjjbbb/TinyImageNet', '/home/ma-user/work/TinyImageNet')

INFO:root:Using MoXing-v1.17.3-
INFO:root:Using OBS-Python-SDK-3.20.7
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 8000
INFO:root:Listing OBS: 9000
INFO:root:Listing OBS: 10000
INFO:root:Listing OBS: 11000
INFO:root:Listing OBS: 12000
INFO:root:Listing OBS: 13000
INFO:root:Listing OBS: 14000
INFO:root:Listing OBS: 15000
INFO:root:Listing OBS: 16000
INFO:root:Listing OBS: 17000
INFO:root:Listing OBS: 18000
INFO:root:Listing OBS: 19000
INFO:root:Listing OBS: 20000
INFO:root:Listing OBS: 21000
INFO:root:Listing OBS: 22000
INFO:root:Listing OBS: 23000
INFO:root:Listing OBS: 24000
INFO:root:Listing OBS: 25000
INFO:root:Listing OBS: 26000
INFO:root:Listing OBS: 27000
INFO:root:Listing OBS: 28000
INFO:root:Listing OBS: 29000
INFO:root:Listing OBS: 30000
INFO:root:Listing OBS: 31000
INFO:root:Listing OBS: 32000
INFO:root:L

In [2]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import torchvision.models as models
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image

In [3]:
my_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

file_path = '/home/ma-user/work/TinyImageNet'

In [21]:
label_num = 100
class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 100 * 1000

    def __getitem__(self, index):
        # print(index)
        label = index % label_num
        img_count = int(index / label_num) % 1000
        # print(f"label : {label}")
        # print(f"img_count : {img_count}")
        img = Image.open(f'{file_path}/train/{label}/{label}_{img_count}.jpg')
        # img.show()
        img = my_transform(img)
        # print(img.size())
        # print(aaa)
        return img, label

    def __len__(self):
        return self.length

In [26]:
train_loader = data.DataLoader(ImageSet(), batch_size=128, shuffle=True)
test_loader = data.DataLoader(ImageSet(), batch_size=1, shuffle=True)

In [23]:
convNet = models.resnet18()
convNet.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
convNet.maxpool = nn.Sequential()
convNet.layer4 = nn.Sequential()
convNet.fc = nn.Linear(256,label_num)
# print(convNet)

In [ ]:
# 加载模型， 请谨慎操作， 会覆盖在内存中的模型
convNet = models.resnet18()
convNet.load_state_dict(torch.load('./ConvNet.model'))
convNet.eval()

In [24]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convNet.parameters(), lr=0.01)


In [27]:
convNet.train()
for i, (images, labels) in enumerate(train_loader):
    outputs = convNet(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    predict = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    print(f"i = {i},  loss = {loss},  accuracy = {float(sum(labels == predict))/float(labels.size(0))}")

i = 0,  loss = 4.393335342407227,  accuracy = 0.015625
i = 1,  loss = 4.507467269897461,  accuracy = 0.0234375
i = 2,  loss = 4.498644828796387,  accuracy = 0.015625
i = 3,  loss = 4.54122257232666,  accuracy = 0.0078125
i = 4,  loss = 4.427181720733643,  accuracy = 0.0234375
i = 5,  loss = 4.297510147094727,  accuracy = 0.0546875
i = 6,  loss = 4.368828773498535,  accuracy = 0.0234375
i = 7,  loss = 4.403121471405029,  accuracy = 0.046875
i = 8,  loss = 4.560285568237305,  accuracy = 0.03125
i = 9,  loss = 4.306641101837158,  accuracy = 0.0546875
i = 10,  loss = 4.414045333862305,  accuracy = 0.0234375
i = 11,  loss = 4.369853496551514,  accuracy = 0.03125
i = 12,  loss = 4.353047847747803,  accuracy = 0.0234375
i = 13,  loss = 4.382089614868164,  accuracy = 0.0390625
i = 14,  loss = 4.288846015930176,  accuracy = 0.0234375
i = 15,  loss = 4.407278060913086,  accuracy = 0.0546875
i = 16,  loss = 4.308373928070068,  accuracy = 0.015625
i = 17,  loss = 4.296656131744385,  accuracy = 0.0

KeyboardInterrupt: 

In [34]:
# 保存模型， 请谨慎操作， 会覆盖文件中的模型
torch.save(convNet.state_dict(), './ConvNet.model')
mox.file.copy_parallel('./ConvNet.model', 'obs://deep-learning-hw2-zzzzzzjjjbbb/ConvNet.model')

''